# Semi-definite Programming
**Note:** This is a summary of what I learnt from reading the first two chapters of _Semidefinite Programming in Quantum Information Science_ by Skrzypczyk & Cavalcanti.
* SDP is a type of convex optimization problem, i.e. the feasible set is convex and thus has only _global_ optima, not local _optima_.
* SDP is a generalization of **linear programming (LP)**. An LP is basically just SDP with diagonal matrices and some other tricks!

#### Primal formulation
$$ \max \text{tr}(A X) $$

subject to
$$ \Phi_i(X) = B_i \quad\quad i = 1,…, m $$
$$ \Gamma_j(X) ≼ C_j \quad\quad j = 1,…, n \quad\quad\quad \text{i.e.} \quad C_j - \Gamma_j(X) \ge 0 $$

Where:
- $X$, the optimisation variable, is a Hermitian operator , i.e. $X^\dagger = X$
- $A, B_i, C_j$ are Hermitian operators.
- $\Phi_i(\cdot)$ and $\Gamma_j(\cdot)$ are linear maps that are hermiticity-preserving, i.e. $\Phi_i(X)$ and
$\Gamma_j(X)$ are Hermitian operators whenever X is Hermitian.

#### Dual formulation
* Every primal SDP has a dual formulation
* The optimal value of the dual problem provides an upper bound on the optimal value of the primal problem.
* **Strong duality** - These optimal values coincide under the mild assumptions of _strict feasibility_ (feasibility even under strict inequality) and _boundedness_.
* If the dual SDP is strictly feasible and bounded, then strong duality also holds.
* It is sufficient to check whether both the primal and dual problems are strictly feasible to have **strong duality**!

$$ \text{minimise} \quad \sum_{i=1}^{m} \text{tr}(Y_i B_i) + \sum_{j=1}^{n} \text{tr}(Z_i C_i) $$
$$ \text{subject to} \quad A - \sum_{i=1}^{m} \Phi_i^{\dagger}(Y_i) - \sum_{j=1}^{n} \Gamma_j^{\dagger}(Z_j) = 0 $$ 
$$ Z_j \succeq 0 \quad j = 1, \dots, n. $$

Where:
- $Y_i$ is the Lagrange multiplier associated with the **equality** constraints on the primal problem.
- $Z_j$ is the Lagrange multiplier associated with the **inequality** constraints on the primal problem.
- Both $Y_i$ and $Z_j$ are Hermitian, i.e. $Y^\dagger_i = Y_i$ and $Z^\dagger_j = Z_j$
---

#### Code examples
From the book by Skrzypczyk & Cavalcanti - _Semidefinite Programming in Quantum Information Science_

In [1]:
## Example 2.1: Maximum Eigenvalue of Hermitian operator
# (Basic example)
import cvxpy as cp
import numpy as np

# Define variables
ρ = cp.Variable((2, 2), symmetric=True)

# Constraints
constraints = [
    ρ >> 0, # positive semi-definite
    cp.trace(ρ) == 1 # trace == 1
]

# Objective function
H = np.array([[0, 1], [1, 0]])
objective = cp.Maximize(cp.trace(H @ ρ))

# Solve SDP
problem = cp.Problem(objective, constraints)
problem.solve()

print("Optimal ρ:\n", ρ.value)
print("tr(Hρ) = ", np.trace(H @ ρ.value))

Optimal ρ:
 [[0.5        0.50000007]
 [0.50000007 0.5       ]]
tr(Hρ) =  1.0000001478211162


In [2]:
## Example 2.2: Hamiltonian feasibility problem
# (Converting feasibility problem into standard problem)
# "The problem is to determine whether there is a Hamiltonian H whose ground-state energy is nonnegative, consistent with the observed data"

# Define variables
λ = cp.Variable((1,1), symmetric=True)
H = cp.Variable((2, 2), symmetric=True)

# Inputs (observed data)
# states measured
ρ = [
    np.array([[0.5, 0], [0, 0.5]]),
    np.array([[0.7, 1.2], [1.2, 0.3]])
]
# observed energy associated with each density matrix
E = [0.2, 0.6] # change one of these to negative to make infeasible

# Constraints
constraints = [
    H >> λ*np.identity(2), # positive semi-definite
    cp.trace(H @ ρ[0]) == E[0], # measuring state gives corresponding energy observed
    cp.trace(H @ ρ[1]) == E[1]
]

# Objective function
objective = cp.Maximize(λ)

# Solve SDP
problem = cp.Problem(objective, constraints)
problem.solve()

is_feasible = "is feasible" if (λ.value >= 0) else "is not feasible"
print("Optimal λ:\n", λ.value)
print("Therefore, the problem of finding a Hamiltonian with non-negative g.s. energy %s" % is_feasible)

Optimal λ:
 [[0.03560894]]
Therefore, the problem of finding a Hamiltonian with non-negative g.s. energy is feasible


In [3]:
## Example 2.3
# Finding trace and operator norms using SDP

dim = 2
A = np.random.rand(dim,dim)

## Calculate norms using numpy:
n1_exp = np.linalg.norm(A, ord=1)
ninf_exp = np.linalg.norm(A, ord=np.inf)

## Using SDP:
# Define variables
x = cp.Variable((1,1), symmetric=True)
X = cp.Variable((dim, dim), symmetric=True)

# Constraints
n1_con = [-X << A, A << X]
ninf_con = [-x*np.identity(dim) << A, A << x*np.identity(dim)]

# Objective function
n1_obj = cp.Minimize(cp.trace(X))
ninf_obj = cp.Minimize(x)

# Solve SDP
n1_sdp = cp.Problem(n1_obj, n1_con)
ninf_sdp = cp.Problem(ninf_obj, ninf_con)
n1_sdp.solve()
ninf_sdp.solve()

# get vals
n1 = np.trace(X.value)
ninf = x.value[0][0]

# compare
print("       Numpy |  SDP  ")
print("  A1 | %.3f | %.3f" % (n1_exp, n1))
print("Ainf | %.3f | %.3f" % (ninf_exp, ninf))

# Observation: SDP not always as effective - numerical inaccuracies


       Numpy |  SDP  
  A1 | 1.117 | 1.219
Ainf | 1.081 | 1.009


In [4]:
## Example 2.4: Dual form of maximum eigenvalue
# input (use the same H as before)
H = np.array([[0, 1], [1, 0]])

y = cp.Variable((1,1), symmetric=True)
constraints = [y*np.identity(2) >> H]

# objective fn
obj = cp.Minimize(y)

# solve SDP
sdp = cp.Problem(obj, constraints)

sdp.solve()

print("Optimal y: ", y.value)
# recall previous Optimal: tr(Hρ) = 1.0000001478211162
# Very similar!!


Optimal y:  [[1.00000098]]
